In [1]:
import torch
from torchtext.datasets import IMDB, AG_NEWS, YahooAnswers
# from torchtext.vocab import GloVe
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torch.nn import LSTM, GRU, Linear, Softmax, CrossEntropyLoss, Conv2d, Dropout
import torch.nn.functional as F
import torch.nn as nn
from torchtext.vocab import GloVe
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split, Dataset
from torch.optim import Adam
from tqdm import tqdm
import numpy as np

In [2]:
from pathlib import Path
from IPython import get_ipython
on_colab = 'google.colab' in str(get_ipython())

if on_colab:
  from google.colab import drive
  drive.mount("/content/gdrive")

PATH =  "/content/gdrive/My Drive/DeepLearning/" if on_colab else "./"
PATH_MODEL = "/content/gdrive/My Drive/DeepLearning/MODELS" if on_colab else "./"

Mounted at /content/gdrive


In [4]:
class BidirectionalLSTMClassifier(torch.nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.LSTM = LSTM(50, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.linear = Linear(2 * hidden_size, num_classes)
        self.softmax = Softmax(dim=1)
        
    def forward(self, x):
        _, (h_n, _) = self.LSTM(x)
        h_forward = h_n[2 * self.num_layers - 2]
        h_backward = h_n[2 * self.num_layers - 1]
        y = self.linear(torch.cat((h_forward, h_backward), 1))
        return self.softmax(y)

In [5]:
num_classes = 2
BATCH_SIZE = 64
SHUFFLE = True
NUM_EPOCHS = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
SOM_distance = 3.1
embedding_path = PATH + 'new_embeddings_IMDB/' + 'new_embeddings_d_' + str(SOM_distance) + '_k_10.pt'
embedding = torch.load(embedding_path)

model = BidirectionalLSTMClassifier(num_classes, 64, 1).to(device)
checkpoint = torch.load(PATH_MODEL + '/' + "LSTM_IMDB_SEM_3.1" + '.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
!pip install textattack[tensorflow]

In [40]:
import textattack
import torchtext

class CustomSEMModelWrapper(textattack.models.wrappers.model_wrapper.ModelWrapper):
    def __init__(self, model, outdim, vocab, device, tokenizer=torchtext.data.utils.get_tokenizer("basic_english")):
        self.model = model
        self.tokenizer = tokenizer
        self.outdim = outdim
        self.vocab = vocab
        self.device = device

    def __call__(self, text_input_list):
        preds = torch.zeros(size=(len(text_input_list), self.outdim))
        for i, review in enumerate(text_input_list):
            tokens = self.tokenizer(review)
            input_list = []
            for _token in tokens:
                embed_temp = self.vocab.get(_token)
                if embed_temp != None:
                    input_list.append(self.vocab.get(_token))
                
                else:
                    input_list.append(torch.zeros(50))

            input_stacked = torch.stack(input_list)
            with torch.no_grad():
                prediction = self.model(torch.unsqueeze(input_stacked, dim=0).to(self.device))
                preds[i] = prediction

        return preds

In [ ]:
model_wrapper = CustomSEMModelWrapper(model, outdim=num_classes, vocab=embedding, device=device)

dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
attack = textattack.attack_recipes.pwws_ren_2019.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(num_examples=200)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class '__main__.BidirectionalLSTMClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 









  0%|          | 0/200 [00:00<?, ?it/s]





  0%|          | 1/200 [00:28<1:35:00, 28.64s/it]





[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:28<1:35:02, 28.66s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (64%)]]

I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them [[crying]]. This movie was great, and I suggest that you go see it before you judge.

I went and saw this movie last night after being coaxed 







[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 2/200 [02:48<4:37:19, 84.04s/it]





[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/200 [02:48<4:37:21, 84.05s/it]

...

--------------------------------------------- Result 16 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (60%)]]

[[I]] [[saw]] this [[film]] at the Rotterdam [[International]] [[Film]] Festival 2002. This seemed to be one of the less popular films on the festival, however, as it [[turned]] out, all the more interesting.<br /><br />The story, of an actor [[trying]] to come to grips with himself and his environment after withdrawing from a drug [[addiction]], is based on actual facts. Moreover, the characters [[playing]] in the film are the real people [[living]] this experience over again, this time for the film, which is partly set up as a stage play. Not only do they all happen to be [[good]] actors, Jia Hongsheng's parents are actors in real life as well, the methods used in highlighting their relationship towards Jia are very effective.<br /><br />Jia Hongsheng is the actor of some Chinese action films late eighties start nineties. Later you can see 







[Succeeded / Failed / Skipped / Total] 16 / 0 / 1 / 17:   9%|▉         | 18/200 [39:16<6:37:03, 130.90s/it]





[Succeeded / Failed / Skipped / Total] 17 / 0 / 1 / 18:   9%|▉         | 18/200 [39:16<6:37:03, 130.90s/it]





[Succeeded / Failed / Skipped / Total] 17 / 0 / 2 / 19:  10%|▉         | 19/200 [39:16<6:14:06, 124.01s/it]

--------------------------------------------- Result 18 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (73%)]]

Even if you're a fan of Jean Rollin's idiosyncratic body of work, you will be caught off guard by this exceptional foray into science fiction territory. For once, there's not a single diaphanously gowned vampire girl in sight ! True to tradition, the budget proved way too tight to realize the director's vision entirely. Yet this is largely compensated by his obvious love of genre cinema, dedication to his craft and sheer ingenuity. Jean-Claude Couty's atmospheric cinematography makes the most of the foreboding locations and Philippe Bréjean (a/k/a "Gary Sandeur") contributes a startling soundtrack that fortunately doesn't resemble any of the sappy stuff he composed for hardcore.<br /><br />Shot in and around a Paris office block before and after working hours, the film was largely cast with porn regulars Rollin was already quite familiar with 







[Succeeded / Failed / Skipped / Total] 17 / 0 / 2 / 19:  10%|█         | 20/200 [39:41<5:57:11, 119.06s/it]





[Succeeded / Failed / Skipped / Total] 18 / 0 / 2 / 20:  10%|█         | 20/200 [39:41<5:57:11, 119.06s/it]

--------------------------------------------- Result 20 ---------------------------------------------
[[Positive (68%)]] --> [[Negative (98%)]]

I went into "Night of the Hunted" not knowing what to expect at all. I was really impressed.<br /><br />It is essentially a mystery/thriller where this girl who can't remember anything gets 'rescued' by a guy who happens to be driving past. The two become fast friends and lovers and together, they try to figure out what is going on with her. Through some vague flashbacks and grim memories, they eventually get to the bottom of it and the ending is pretty cool.<br /><br />I really liked the setting of this one: a desolate, post-modern Paris is the backdrop with lots of gray skies and [[tall]] buildings. Very metropolitan. Groovy soundtrack and lots of nudity.<br /><br />Surprising it was made in 1980; seems somewhat ahead of it's time.<br /><br />8 out of 10, kids.

I went into "Night of the Hunted" not knowing what to expect at all. I was reall